<h4 >Mahdi Abbariki</h4>
<h5>983212001</h5>

objective: <br>
    minimizing equation using ES algorithm

First we import methods and libraries we need

In [283]:
from numpy import power
from numpy import sum
from numpy.random import uniform
from numpy.random import normal
from numpy import argsort


define a method to evaluate equation

In [284]:
def objective(x,n):
    powers = power(x[:n], 2)
    return sum(powers)

define a method to check if a generated candidate is in the specified bounds or not

In [285]:
def in_bounds(point, bounds):
    for i in range(len(point)):
        item = point[i]
        if item < bounds[0] or item > bounds[1]:
            return False
    return True

create initial population

In [286]:
def initPopulation(lam,bounds,n):
    population = list()
    for _ in range(lam):
        candidate = None
        while candidate is None or not in_bounds(candidate, bounds):
            candidate = []
            for _ in range(n):
                candidate.append(uniform(low=bounds[0], high=bounds[1]))
        population.append(candidate)
    return population

now we can create the esPlus method to deal with searching

In [287]:
def esPlus(mu,lam,maxIter,objective,bounds,n):
    best, best_eval = None, 1e+100
    n_children = int(lam / mu)
    population = initPopulation(lam,bounds,n)
    normalScale = 1
    normalScaleC = 0.9

    # perform the search
    for epoch in range(maxIter):
	    # evaluate fitness for the population
        scores = [objective(c,n) for c in population]
        # rank scores in ascending order
        ranks = argsort(argsort(scores))
        # select the indexes for the top mu ranked solutions
        selected = [i for i,_ in enumerate(ranks) if ranks[i] < mu]

        #create children
        children = []
        for i in selected:
            #update best and best eval based on parent
            if scores[i] < best_eval:
                best, best_eval = population[i], scores[i]
           
            # keep the parent
            children.append(population[i])
            #count better children for calculating normalScare dynamically
            betterChild = 0
            # create children for parent
            for _ in range(n_children):
                child = None
                while child is None or not in_bounds(child,bounds):
                    child = []
                    for j in range(n):
                        child.append(population[i][j] + normal(loc=0, scale=normalScale))
                if(objective(child,n) < objective(population[i],n)): # child is better than parent
                    betterChild += 1
                children.append(child)

        # evaluate new scale for gaussian distribution
        ps = betterChild / n_children
        if(ps < 1/5):
            normalScale *= normalScaleC
        elif(ps > 1/5):
            normalScale /= normalScaleC

        # replace current populations with children
        population = children
    return [best,best_eval]

specify the inputs and run the algorithm

In [288]:
bounds = [-100,100]

maxIterations = 100000

mu = 1

lam = 1

n = 100

best, score = esPlus(mu,lam,maxIterations,objective,bounds,n)
print('Done!')
print('f(%s) = %f' % (best, score))


Done!
f([-13.444982152235841, 7.278129690239458, 7.416441558529936, 1.9528036140676854, -3.085967102919726, 8.013942573162385, -4.23879298596588, -9.243668063474738, 5.7653191543153675, 5.472907950182471, -15.974630982962049, -7.417608696821107, -12.569478834977781, 6.713303583246035, 10.661050095703333, -9.58198486539092, 6.240654500256637, 3.010352442909869, 7.70782981917651, -4.852212477457648, 11.466084861892416, 7.328815468963529, 4.756443876392413, -6.370574771079475, 9.513545514170119, 8.198871546666126, 17.72577583357431, -11.267082402188258, 2.6987852818227505, 9.171258253482296, -21.507864615287218, -7.806373871548458, -17.674874672251608, 4.486555742212781, 22.616168000638737, 12.672123445068115, -13.530508796175521, -2.7420342520379455, 23.65071505953753, 12.395482766238938, -14.19997810392054, -11.59826464881041, 12.109299315004538, 1.9393735550074556, -8.787188803423557, -2.945030112399821, -5.730013721883973, 8.745427624193132, 5.633616767565129, 11.336953352662851, 18.8